<a href="https://colab.research.google.com/github/sahilaf/Test_models/blob/main/Preprocess_gstalker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Make directories

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir -p Obama_data/{ori_imgs,parsing,torso_imgs}

# Convert Original frames from the original video

In [ ]:
!ffmpeg -i my_video.mov -qscale:v 2 Obama_data/ori_imgs/%05d.jpg

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
!mv /content/drive/MyDrive/Obama_data /content/

In [ ]:
!mv /content/output/my_video.csv /content/Obama_data/

In [ ]:
!cp -r /content/AD-NeRF/ /content/drive/MyDrive/

In [ ]:
!apt-get install imagemagick -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript gsfonts
  imagemagick-6-common imagemagick-6.q16 libdjvulibre-text libdjvulibre21
  libfftw3-double3 libgs9 libgs9-common libidn12 libijs-0.35 libjbig2dec0
  libjxr-tools libjxr0 liblqr-1-0 libmagickcore-6.q16-6
  libmagickcore-6.q16-6-extra libmagickwand-6.q16-6 libnetpbm10
  libwmflite-0.2-7 netpbm poppler-data
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre
  ghostscript-x imagemagick-doc autotrace cups-bsd | lpr | lprng enscript gimp
  gnuplot grads hp2xx html2ps libwmf-bin mplayer povray radiance sane-utils
  texlive-base-bin transfig ufraw-batch libfftw3-bin libfftw3-dev inkscape
  poppler-utils fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-gothic fonts-arphic-ukai
  fonts-arphic-uming fon

In [ ]:
!mogrify -format jpg Obama_data/gt_imgs/*.bmp

In [ ]:
!ls

au.csv	gt_imgs  Obama_data.mp4  ori_imgs  parsing  torso_imgs


In [ ]:
!rm ori_imgs/*.jpg

In [ ]:
%cd Obama_data

/content/Obama_data


In [ ]:
!ls

au.csv	gt_imgs  Obama_data.mp4  ori_imgs  parsing  torso_imgs


In [ ]:
import pandas as pd, numpy as np, os, json

csv = pd.read_csv('au.csv')
out_dir = 'ori_imgs'
os.makedirs(out_dir, exist_ok=True)

for i, row in csv.iterrows():
    lms = np.array([[row[f'x_{j}'], row[f'y_{j}']] for j in range(68)])
    np.savetxt(os.path.join(out_dir, f"{i}.lms"), lms)


In [ ]:
import os

folder = "/content/Obama_data/ori_imgs"  # change if your path differs
files = sorted(os.listdir(folder))  # will correctly sort 00001.jpg, 00002.jpg, etc.

# filter only image files
files = [f for f in files if f.lower().endswith('.jpg')]

for i, filename in enumerate(files):
    src = os.path.join(folder, filename)
    dst = os.path.join(folder, f"{i}.jpg")
    os.rename(src, dst)

print(f"✅ Renamed {len(files)} files from 0.jpg to {len(files)-1}.jpg")


✅ Renamed 892 files from 0.jpg to 891.jpg


In [ ]:
import os
import re

folder = "/content/drive/MyDrive/Obama_data/gt_imgs"

# Function to extract number from filename
def get_number(filename):
    nums = re.findall(r'\d+', filename)
    return int(nums[0]) if nums else -1

# Get only .jpg files and sort them numerically
files = sorted(
    [f for f in os.listdir(folder) if f.lower().endswith('.jpg')],
    key=get_number
)

# Rename sequentially
for i, filename in enumerate(files):
    src = os.path.join(folder, filename)
    dst = os.path.join(folder, f"{i}.jpg")
    os.rename(src, dst)

print(f"✅ Renamed {len(files)} files numerically from 0.jpg to {len(files)-1}.jpg")


✅ Renamed 892 files numerically from 0.jpg to 891.jpg


# Parsing

In [2]:
!git clone https://github.com/YudongGuo/AD-NeRF.git
%cd AD-NeRF

Cloning into 'AD-NeRF'...
remote: Enumerating objects: 161, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 161 (delta 21), reused 19 (delta 19), pack-reused 133 (from 1)
Receiving objects: 100% (161/161), 156.66 MiB | 12.43 MiB/s, done.
Resolving deltas: 100% (60/60), done.
/content/AD-NeRF


In [3]:
!pip install imageio[ffmpeg] scipy opencv-python numpy tqdm

In [ ]:
!mkdir -p data_util/face_tracking/3DMM
!wget https://github.com/wuhaozhe/AD-NeRF/raw/main/data_util/face_tracking/3DMM/BFM_model_front.mat -O data_util/face_tracking/3DMM/01_MorphableModel.mat

--2025-10-12 10:54:22--  https://github.com/wuhaozhe/AD-NeRF/raw/main/data_util/face_tracking/3DMM/BFM_model_front.mat
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-10-12 10:54:22 ERROR 404: Not Found.



In [ ]:
!ls

dataset    environment.yml  NeRFs	pretrained_models  README.md
data_util  LICENSE	    paper_data	process_data.sh


In [7]:
!ls -lh /content/AD-NeRF/data_util/face_tracking/3DMM/01_MorphableModel.mat


-rw-r--r-- 1 root root 230M Oct 12 16:26 /content/AD-NeRF/data_util/face_tracking/3DMM/01_MorphableModel.mat


In [8]:
!ls

AD-NeRF  drive	sample_data


In [9]:
%cd AD-NeRF/

/content/AD-NeRF


In [10]:
%cd data_util/face_tracking
!python convert_BFM.py
%cd ../../


/content/AD-NeRF/data_util/face_tracking
/content/AD-NeRF


In [4]:
!ls


dataset    environment.yml  NeRFs	pretrained_models  README.md
data_util  LICENSE	    paper_data	process_data.sh


In [5]:
%cd ..

/content


In [6]:
!ls -lh 01_MorphableModel.mat

-rw-r--r-- 1 root root 230M Oct 12 16:26 01_MorphableModel.mat


In [13]:
!pip install face-alignment

  Using cached face_alignment-1.4.1-py2.py3-none-any.whl.metadata (7.4 kB)
Using cached face_alignment-1.4.1-py2.py3-none-any.whl (30 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 149.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 165.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 59.3 MB/s eta 0:00:00


In [14]:
!pip install resampy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 99.3 MB/s eta 0:00:00


In [5]:
import torch
print(torch.__version__)
print(torch.version.cuda)

2.3.1+cu121
12.1


In [5]:
!git clone https://github.com/facebookresearch/pytorch3d.git

Cloning into 'pytorch3d'...
remote: Enumerating objects: 15121, done.
remote: Counting objects: 100% (2695/2695), done.
remote: Compressing objects: 100% (377/377), done.
remote: Total 15121 (delta 2432), reused 2326 (delta 2318), pack-reused 12426 (from 1)
Receiving objects: 100% (15121/15121), 49.41 MiB | 12.09 MiB/s, done.
Resolving deltas: 100% (10613/10613), done.


In [6]:
!cd pytorch3d && pip install -e .

Obtaining file:///content/pytorch3d
  Preparing metadata (setup.py) ... done
  DEPRECATION: Legacy editable install of pytorch3d==0.7.8 from file:///content/pytorch3d (setup.py develop) is deprecated. pip 25.0 will enforce this behaviour change. A possible replacement is to add a pyproject.toml or enable --use-pep517, and use setuptools >= 64. If the resulting installation is not behaving as expected, try using --config-settings editable_mode=compat. Please consult the setuptools documentation for more information. Discussion can be found at https://github.com/pypa/pip/issues/11457
  Running setup.py develop for pytorch3d


In [15]:
!pip install python_speech_features

  Preparing metadata (setup.py) ... done
  Created wheel for python_speech_features: filename=python_speech_features-0.6-py3-none-any.whl size=5868 sha256=5cb6c03e21a051e8cb8735309651d1be435af9956278644fd9c83eb53dcba921
  Stored in directory: /root/.cache/pip/wheels/60/90/3c/4b5996a95d363fa14525597a19146a940bec467b44b2a14580
Successfully built python_speech_features


In [8]:
!ls

AD-NeRF  condacolab_install.log  drive	pytorch3d  sample_data


In [9]:
%cd AD-NeRF/

/content/AD-NeRF


In [11]:
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 65.3 MB/s eta 0:00:00


In [15]:
!pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 147.3 MB/s eta 0:00:00


In [16]:
!pip install configargparse

In [20]:
!python data_util/process_data.py --id Obama --step 0

--- Step0: extract deepspeech feature ---
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab

In [21]:
!python data_util/process_data.py --id Obama --step 1

--- Step1: extract images from vids ---


In [23]:
!python data_util/process_data.py --id Obama --step 2

--- Step 2: detect landmarks ---
Downloading: "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" to /root/.cache/torch/hub/checkpoints/s3fd-619a316812.pth
100% 85.7M/85.7M [00:06<00:00, 13.3MB/s]
Downloading: "https://www.adrianbulat.com/downloads/python-fan/2DFAN4-cd938726ad.zip" to /root/.cache/torch/hub/checkpoints/2DFAN4-cd938726ad.zip
100% 91.9M/91.9M [00:05<00:00, 16.2MB/s]


In [24]:
!python data_util/process_data.py --id Obama --step 3

--- Step 3: face parsing ---
Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth
100% 44.7M/44.7M [00:00<00:00, 403MB/s]
processed parsing 100
processed parsing 200
processed parsing 300
processed parsing 400
processed parsing 500
processed parsing 600
processed parsing 700
processed parsing 800
processed parsing 900
processed parsing 1000
processed parsing 1100
processed parsing 1200
processed parsing 1300
processed parsing 1400
processed parsing 1500
processed parsing 1600
processed parsing 1700
processed parsing 1800
processed parsing 1900
processed parsing 2000
processed parsing 2100
processed parsing 2200
processed parsing 2300
processed parsing 2400
processed parsing 2500
processed parsing 2600
processed parsing 2700
processed parsing 2800
processed parsing 2900
processed parsing 3000
processed parsing 3100
processed parsing 3200
processed parsing 3300
processed parsing 3400
processed parsing 3500
pr

In [25]:
!python data_util/process_data.py --id Obama --step 4

--- Step 4: extract background image ---
(400, 202500, 1)
(132581, 2)
449 0


In [26]:
!python data_util/process_data.py --id Obama --step 5

--- Step 5: save training images ---


In [ ]:
!python data_util/process_data.py --id Obama --step 6

--- Estimate Head Pose ---
600 2.08974289894104 -3.9530394077301025
700 2.0658817291259766 -4.618486404418945
800 2.0533390045166016 -5.28143310546875
900 2.0461668968200684 -5.982255935668945
1000 2.0434632301330566 -6.702089786529541
1100 2.0359456539154053 -7.3467698097229
1200 2.037019968032837 -8.085104942321777
1300 2.0394115447998047 -8.830256462097168
1400 2.042193651199341 -9.599061965942383
find best focal 1100
2.0733931064605713 -7.6799821853637695
0 of 160 done
1 of 160 done
2 of 160 done
3 of 160 done
4 of 160 done
5 of 160 done
6 of 160 done
7 of 160 done
8 of 160 done
9 of 160 done
10 of 160 done
11 of 160 done
12 of 160 done
13 of 160 done
14 of 160 done
15 of 160 done
16 of 160 done
17 of 160 done
18 of 160 done
19 of 160 done
20 of 160 done
21 of 160 done
22 of 160 done
23 of 160 done
24 of 160 done
25 of 160 done
26 of 160 done
27 of 160 done
28 of 160 done
29 of 160 done
30 of 160 done
31 of 160 done


In [ ]:
!python data_util/process_data.py --id Obama --step 7